# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Answer: This is a classsification problem because the output is binary: either we need to intervene or we don't. If we needed to identify how much intervention a student needed (and that variable was continuous), we would use a regressor. In its current state the problem is one of classification.


## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [5]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

IOError: File student-data.csv does not exist

Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [5]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
n_students = student_data.shape[0]
n_features = student_data.shape[1] - 1
n_passed = student_data[student_data['passed']=='yes'].shape[0]
n_failed = student_data[student_data['passed']=='no'].shape[0]
grad_rate = n_passed*100/float(n_students)
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 265
Number of students who failed: 130
Number of features: 30
Graduation rate of the class: 67.09%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [6]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [7]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [8]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test/float(num_all)) 
print num_test/float(num_all)

print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

0.240506329114
Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

- What are the general applications of this model? What are its strengths and weaknesses?
- Given what you know about the data so far, why did you choose this model to apply?
- Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

## Answer: Model 1: Decision Tree classifier

-General Applications: regression and classification problems. Often used for data mining

-Strengths: Easy to visualize and implement, inexpensive to compute, can handle numeric and categorical data

-Weaknesses: tendency to overfit, certain concepts are hard to learn, can create biased trees without balanced data

I chose this model to start because it is easy to implement, understand and visaulize. It seems to be an excellent model for starting out with the data. 

Sources:http://scikit-learn.org/stable/modules/tree.html



In [21]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Done!\nTraining time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5)

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002


In [22]:
# Predict on training set and compute F1 score
from sklearn.metrics import f1_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Done!\nPrediction time (secs): {:.3f}".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.878260869565


In [23]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.796992481203


In [24]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    print "Training set size: {}".format(len(X_train))
    train_classifier(clf, X_train, y_train)
    print "F1 score for training set: {}".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

# Run the helper function above for desired subsets of training data
#300: same as above
train_predict(clf, X_train, y_train, X_test, y_test)
#200
X_train_200=X_train.sample(n=200)
y_train_200=y_train.sample(n=200)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
#100
X_train_100=X_train.sample(n=100)
y_train_100=y_train.sample(n=100)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)


# Note: Keep the test set constant

------------------------------------------
Training set size: 300
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.002
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.878787878788
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.80303030303
------------------------------------------
Training set size: 200
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.891228070175
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.666666666667
------------------------------------------
Training set size: 100
Training DecisionTreeClassifier...
Done!
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Done!
Prediction time (secs): 0.000
F1 sco

| SampleSize  |  100 | 200  |  300 |
|---|---|---|---|
| TrainingTime   |  .001 |  .001 |  .002 |
| PredictionTime  | .000  |  .000 | .001  |
| F1 Score Training  | .881  | .891 |  .879 |
| F1 Score Testing  | .77  |  .667 |  .803 |

## Answer: Model 2: SVM

-General Applications: regression, classification, and outlier detection. Often used for prediction and generating lines of best fit

-Strengths: Effective with lots of features, versitile using the kernel trick, memory efficient due to SV's

-Weaknesses: No direct probability estimate, if features are greater than samples accuracy suffers

I chose this model as my second because it is another model which is easy to understand and implement. It will make a good comparision for the Decision Tree Classifier.

Source:http://scikit-learn.org/stable/modules/svm.html#svm

In [25]:
#model 2 Training
from sklearn import svm
clf = svm.SVC()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training SVC...
Done!
Training time (secs): 0.007


In [26]:
# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using SVC...
Done!
Prediction time (secs): 0.005
F1 score for training set: 0.860706860707


In [27]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using SVC...
Done!
Prediction time (secs): 0.003
F1 score for test set: 0.780821917808


In [1]:
# Run the helper function above for desired subsets of training data
#300: same as above
train_predict(clf, X_train, y_train, X_test, y_test)
#200
X_train_200=X_train.sample(n=200)
y_train_200=y_train.sample(n=200)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
#100
X_train_100=X_train.sample(n=100)
y_train_100=y_train.sample(n=100)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)


# Note: Keep the test set constant

NameError: name 'train_predict' is not defined

| SampleSize  |  100 | 200  |  300 |
|---|---|---|---|
| TrainingTime   |  .001 |  .003 |  .007 |
| PredictionTime  | .001  | .002  | .006  |
| F1 Score Training  |  .836 | .831  | .861  |
| F1 Score Testing  |  .738 | .758  | .78  |

## Answer: Model 3: Neural Net -not implemented

-General Applications:  function estimation 

-Strengths: can be used for deep learning, can learn non-linear models, can simulate kernel trick in SVM without handcrafted kernel

-Weaknesses: many hyperparameters (could also be a strength) , tendency to overfit to local minima may lead to different scores with different random starting weights <--(this can be solved using more dimensions as detailed here:http://arxiv.org/pdf/1412.0233.pdf)

I chose this model as my third because though it is harder to implement, it should be more accurate than the previous two. 

NOTE: MLPCLassifier is not implemented in sklearn v0.17 Please see model 4 for an implementation of naive bayes instead.






## Answer: Model 4: Naive Bayes

-General Applications:  Classification assuming vars follow iid. commonly used in spam detection

-Strengths: decent classifier, easy to implement and add to

-Weaknesses: bad estimator, assumes vars follow iid, if a feature and label pair do not exist in the sample Naive Bayes will assume a probability of 0 unless we do some sample correction. 

I chose this model because this is a classification problem and Naive Bayes is a decent classifier. In addition it seems that most features would be normally distributed and a large percentage would be independent.


Sources:http://scikit-learn.org/stable/modules/naive_bayes.html



In [29]:
#model 34 Training
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training GaussianNB...
Done!
Training time (secs): 0.002


In [30]:
# Predict on training set and compute F1 score
train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.799054373522


In [31]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.758064516129


In [32]:
# Run the helper function above for desired subsets of training data
#300: same as above
train_predict(clf, X_train, y_train, X_test, y_test)
#200
X_train_200=X_train.sample(n=200)
y_train_200=y_train.sample(n=200)
train_predict(clf, X_train_200, y_train_200, X_test, y_test)
#100
X_train_100=X_train.sample(n=100)
y_train_100=y_train.sample(n=100)
train_predict(clf, X_train_100, y_train_100, X_test, y_test)


# Note: Keep the test set constant

------------------------------------------
Training set size: 300
Training GaussianNB...
Done!
Training time (secs): 0.002
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.001
F1 score for training set: 0.799054373522
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.758064516129
------------------------------------------
Training set size: 200
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.771428571429
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for test set: 0.705882352941
------------------------------------------
Training set size: 100
Training GaussianNB...
Done!
Training time (secs): 0.001
Predicting labels using GaussianNB...
Done!
Prediction time (secs): 0.000
F1 score for training set: 0.821705426357
Predicting labels using GaussianNB...
Done!
Prediction time

| SampleSize  |  100 | 200  |  300 |
|---|---|---|---|
| TrainingTime   | .001  | .001  | .002  |
| PredictionTime  | .000  | .000  | .001  |
| F1 Score Training  | .822  | .771  | .799  |
| F1 Score Testing  | .651  | .706  | .758  |

## 5. Choosing the Best Model

- Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?
- In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).
- Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.
- What is the model's final F<sub>1</sub> score?

**ANSWER:**

Given the performance of the models and the nature of our problem, I believe SVM would be the proper model choice based on performance. Its accuracy is consistently high and it is inexpensive as long as we don't require probabilities. The Naive Bayes is a decent classifier but its accuracy is consistenty lower than the SV, especially as the datasize shrinks. The continuous nature of the model features means that the SVM makes more logical sense than decision tree. Though the decision tree has a higher F score on the training size of 300, I believe that robustness of the SVM's results and the fact that it has no parameters yet make it the better choice.
Even though the SVM is not cheap compared with the other 2 models the cost of this project will be low as we are only looking at one school district. I believe in this case we should value performance over relative cost as we are realistically talking about a very small amount of money. 

NOTE: In my decisions tree example I specified a max depth of 5 in order to make a more informed decision on the viabiliity of the model.

An SVM first transforms the data of the model into a number of dimensions specified by the user. It does this in order to be able to apply functions to nonlinearly-sperable data. This is performed using the kernel trick. Once the kernel function has been applied the SVM fits itself to the data by minimizing the length of the weights vector or ||w||.. This minimization is essentially equivalent to maximizing the margin and reducing error.

For the layperson this means that we look at the data and we draw classification lines that maximize the distance from the lines to the points. If the data cannot be separated by lines we transform it until we can see a way to classify the data using lines. THis transformation is done using a class of functions called kernels. 



In [41]:
# TODO: Fine-tune your model and report the best F1 score
from sklearn import grid_search
from sklearn.metrics import classification_report
from sklearn.metrics import fbeta_score, make_scorer

parameters = {'kernel':('linear', 'rbf'), 'C':[1,2,3,4], 'degree':[3,4,5]}
svr = svm.SVC()
def performance_metric(label, prediction):
    return f1_score(label, prediction, pos_label='yes')

scorer = make_scorer(performance_metric)
clf = grid_search.GridSearchCV(svr, parameters, scorer)
train_predict(clf, X_train, y_train, X_test, y_test)
print clf.best_estimator_.kernel
print clf.best_estimator_.C
print clf.best_estimator_.degree

------------------------------------------
Training set size: 300
Training GridSearchCV...
Done!
Training time (secs): 2.565
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.004
F1 score for training set: 0.860706860707
Predicting labels using GridSearchCV...
Done!
Prediction time (secs): 0.001
F1 score for test set: 0.780821917808
rbf
1
3


**ANSWER:**The final score on the test set for the model after optimization is: 0.78

This matches the orginal SVM's score on the test set. Indeed if we look at the optimally chosen SVM's HyperParameters we can see that they match the default